# Using panel for dashboards in python


In [57]:
import panel as pn
import pandas as pd
import panda_bokeh
#from hvplot import hvPlot
#import holoviews as hv
#hv.extension('bokeh')
#import hvplot.pandas

pn.extension()

ModuleNotFoundError: No module named 'panda_bokeh'

In [56]:
def f(x):
    return x * x
pn.interact(f, x=10)

Column
    [0] Column
        [0] IntSlider(end=30, name='x', start=-10, value=10, value_throttled=10)
    [1] Row
        [0] Str(int, name='interactive00307')

In [15]:
df = pd.read_excel('FloodArchive.xlsx')
df.head()

,ID,GlideNumber,Country,OtherCountry,long,lat,Area,Began,Ended,Validation,Dead,Displaced,MainCause,Severity
0,1,0,Algeria,0,5.23026,35.81420,92615.67,1985-01-01,1985-01-05,News,26,3000,Heavy rain,1.0
1,2,0,Brazil,0,-45.34890,-18.71110,678498.82,1985-01-15,1985-02-02,News,229,80000,Heavy rain,2.0
2,3,0,Phillipines,0,122.97400,10.02070,12846.03,1985-01-20,1985-01-21,News,43,444,Torrential rain,1.0
3,4,0,Indonesia,0,124.60600,1.01489,16542.12,1985-02-04,1985-02-18,News,21,300,Torrential rain,1.0
4,5,0,Mozambique,0,32.34910,-25.86930,20082.21,1985-02-09,1985-02-11,News,19,0,Heavy rain,2.0


In [22]:
year = pn.widgets.IntRangeSlider(name = 'Years Range Slider', width=250, start=1985, end=2016, value=(1985, 2016), value_throttled=(1985, 2016))
year

IntRangeSlider(end=2016, name='Years Range Slider', start=1985, value=(1985, 2016), value_throttled=(1985, 2016), width=250)

In [23]:
@pn.depends(year.param.value_throttled)
def year_range(year):
    return '### Floods Between {start} —  {end}'.format(start=year[0], end=year[1])
pn.Row(year_range)

Row
    [0] ParamFunction(function)

In [25]:
# KPI — Death
df['Began'] = pd.to_datetime(df['Began'])
df['Ended'] = pd.to_datetime(df['Ended'])
@pn.depends(year.param.value_throttled)
def dead_stat(year):
    return '### Death {}'.format(df[df.Began.dt.year.between(year[0], year[1])]['Dead'].sum())
pn.Row(dead_stat)

Row
    [0] ParamFunction(function)

In [46]:
# Bar chart
@pn.depends(year.param.value_throttled)
def plot_bar(year):
    years_df = df[df.Began.dt.year.between(year[0], year[1])]
    bar_table = years_df['MainCause'].value_counts().reset_index().rename(columns={'index':'Cause', 'MainCause':'Total'})
    return bar_table[:10].sort_values('Total').hvplot.barh('Cause', 'Total', invert=False, legend='bottom_right', height=600)

In [47]:
# Map
@pn.depends(year.param.value_throttled)
def plot_map(year):
    years_df = df[df.Began.dt.year.between(year[0], year[1])]
    return df.hvplot.points('long', 'lat', xaxis=None, yaxis=None, geo=True, color='red', alpha=0.5, tiles='CartoLight', hover_cols=["Began", "Ended", "Dead", "Displaced"], size=5)

In [48]:


title = '# Worldwide Floods'
logo = pn.panel('floodsImage.png', width=200, align='start')
text = 'Dataset:Global Active Archive of Large Flood Events '# Header box
header_box = pn.WidgetBox(title, logo, year,  pn.layout.Spacer(margin=200), text, width=300, height=1000, align="center")# Plot Box
plots_box = pn.WidgetBox(pn.Column(pn.Row(pn.Column(year_range), pn.layout.HSpacer(), pn.Column(dead_stat)), plot_map, plot_bar, align="start", width=600,sizing_mode="stretch_width"))# Dashboard
dashboard = pn.Row(header_box, plots_box, sizing_mode="stretch_width")
dashboard

AttributeError: 'DataFrame' object has no attribute 'hvplot'